In [1]:
from datasets.load import loadDataset
#dataset = loadDataset('_filtered3months_indicators', combineObsInd=True,withActions=False)
dataset = loadDataset('_filtered3months_indicators', combineObsInd=False,withActions=True)

from utils.medical import cleanLabels
patient_dict =  cleanLabels(dataset['patient_dictionary'],'_filtered3months_indicators') 
import numpy as np
%matplotlib inline  
import glob,h5py,os,re
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['lines.linewidth']=2.5
mpl.rcParams['lines.markersize']=8
mpl.rcParams['text.usetex']=True
mpl.rcParams['text.latex.unicode']=True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'
mpl.rcParams['text.latex.preamble']= ['\usepackage{amsfonts}','\usepackage{amsmath}']
mpl.rcParams['font.size'] = 20
mpl.rcParams['axes.labelsize']=20
mpl.rcParams['legend.fontsize']=20
#http://stackoverflow.com/questions/22408237/named-colors-in-matplotlib

Found medical files...
Data Shape for Obs|Act|Ind:  (8244, 18, 48) (8244, 18, 8) (8244, 18, 4)
<<< Train (5276)/Valid (1319)/Test (1649) >>>
<<<Seperate indicators and observations>>>
<<<Using actions>>>


In [8]:
from models.temporal_medical import DKF
#Load model to sample from
#cat
MFILE = '../../expt-medical/chkpt_filtered3months_indicators-cfac/DKF_lr-8_0000e-04-vm-lstm-inf-mean_field-dh-200-ds-30-nl-relu-opt-adam-bs-250-ep-2000-rd-2_0000e-01-rs-300-tl-2-el-2-ttype-mlp-etype-mlp-ar-1_0000e+01-iw-1_0000e-01-sum'

#sum
#MFILE = '../../expt-medical/chkpt_filtered3months_indicators-cfac/DKF_lr-8_0000e-04-vm-lstm-inf-mean_field-dh-200-ds-30-nl-relu-opt-adam-bs-250-ep-2000-rd-2_0000e-01-rs-300-tl-2-el-2-ttype-mlp-etype-mlp-ar-1_0000e+01-iw-1_0000e-01-sum'
reloadFile = MFILE+'-EP1900-params.npz'
pfile      = MFILE+'-config.pkl'
print 'Reloading trained model from : ',reloadFile
print 'Assuming ',pfile,' corresponds to model'
params= {}
params['seed'] = 1
params['validate_only'] = True
dkf  = DKF(params, paramFile = pfile, reloadFile = reloadFile)

Reloading trained model from :  ../../expt-medical/chkpt_filtered3months_indicators-cfac/DKF_lr-8_0000e-04-vm-lstm-inf-mean_field-dh-200-ds-30-nl-relu-opt-adam-bs-250-ep-2000-rd-2_0000e-01-rs-300-tl-2-el-2-ttype-mlp-etype-mlp-ar-1_0000e+01-iw-1_0000e-01-sum-EP1900-params.npz
Assuming  ../../expt-medical/chkpt_filtered3months_indicators-cfac/DKF_lr-8_0000e-04-vm-lstm-inf-mean_field-dh-200-ds-30-nl-relu-opt-adam-bs-250-ep-2000-rd-2_0000e-01-rs-300-tl-2-el-2-ttype-mlp-etype-mlp-ar-1_0000e+01-iw-1_0000e-01-sum-config.pkl  corresponds to model
		<< Reloading Model >>
		<< Loading structure (../../expt-medical/chkpt_filtered3months_indicators-cfac/DKF_lr-8_0000e-04-vm-lstm-inf-mean_field-dh-200-ds-30-nl-relu-opt-adam-bs-250-ep-2000-rd-2_0000e-01-rs-300-tl-2-el-2-ttype-mlp-etype-mlp-ar-1_0000e+01-iw-1_0000e-01-sum-config.pkl) and model (../../expt-medical/chkpt_filtered3months_indicators-cfac/DKF_lr-8_0000e-04-vm-lstm-inf-mean_field-dh-200-ds-30-nl-relu-opt-adam-bs-250-ep-2000-rd-2_0000e-01-r

In [3]:
def patientEligibility(U, docdict, drugName = 'Metformin'):
    """
    Return a list with tuples of (patient index, stop_t)
    After that do inference up to point stop_t and forward sample from then on until Tmax [=18]
    """
    drugIdx = -1
    for idx,d in enumerate(docdict):
        if drugName.lower() in d.lower():
            drugIdx = idx
            break
    assert drugIdx>=0,'Drug not found in list: '+drugName
    print 'Halting at first incidence of '+drugName+' at index= '+str(drugIdx)
    assert U.shape[2]==len(docdict),'Shape of doctor dictionary does not match up'
    plist = []
    Tmax = U.shape[1]
    for idx in range(U.shape[0]):
        patient = U[idx,:,:]
        #Find first incidence of drug
        incidences= np.where(patient[:,drugIdx])[0]
        if len(incidences)==0:
            continue #No Metformin found
        else:
            firstPrescription = incidences[0]
            #Check if any other drugs prescribed
            otherDrugs        = patient[:,:drugIdx].sum()
            if otherDrugs>0 or firstPrescription<1 or firstPrescription>Tmax-2:
                continue
            plist.append((idx,firstPrescription))
    print 'Processed Test Patients: ',len(plist),' patients suitable for cfac analysis'
    return plist
    
#First determine subset of patients based on actions
test_X    = dataset['test_X']
test_U    = dataset['test_U']
test_I    = dataset['test_I']
mask_test = dataset['mask_test_X']
testplist = patientEligibility(test_U, dataset['doctor_dictionary'])


Halting at first incidence of Metformin at index= 6
Processed Test Patients:  666  patients suitable for cfac analysis


In [4]:
maxT      = test_X.shape[1]

FAC_Z = []
CFAC_Z= []
#We need to also track this so that when we do reconstruction at the end,we're OK
CFAC_I= [] 
for idx,(pid,stop_t) in enumerate(testplist):
    X_infer= test_X[[pid],:stop_t,:]
    U_infer= test_U[[pid],:stop_t,:]
    I_infer= test_I[[pid],:stop_t,:]
    if idx%100==0:
        doPrint = True
        print '#:',idx
    else:
        doPrint = False
    if doPrint:
        print 'Input: ',X_infer.shape,U_infer.shape,I_infer.shape,stop_t
    #Run posterior inference
    z_infer,mu_infer,_ = dkf.infer(X_infer,U=U_infer,I=I_infer)
    
    if doPrint:
        print 'Inference',mu_infer.shape
    #Perform cfac inference
    #Start from the same mean
    z_fact         = np.copy(mu_infer[:,[-1],:])
    z_cfact        = np.copy(mu_infer[:,[-1],:])
    
    #Track latent states
    all_zs_fact = []
    all_zs_cfact= []
    
    #For the remainder of time
    for t in range(stop_t,maxT):
        factU      = test_U[[pid],[t],:].astype('float32').reshape(1,1,test_U.shape[2])
        #factU      = np.ones(factU.shape).astype('float32')
        cfactU     = np.zeros(factU.shape).astype('float32')
        #Apply transition function
        mu_fact,logcov_fact  = dkf.transition_fxn(z_fact,U=factU)
        mu_cfact,logcov_cfact = dkf.transition_fxn(z_cfact,U=cfactU)
        all_zs_fact.append(np.copy(mu_fact))
        all_zs_cfact.append(np.copy(mu_cfact))
        #For the next iteration use the means
        z_fact     = np.copy(mu_fact)
        z_cfact    = np.copy(mu_cfact)
    latent_z_fact  = np.concatenate(all_zs_fact,axis=1)
    latent_z_cfact  = np.concatenate(all_zs_cfact,axis=1)
    
    CFAC_I.append(np.concatenate([test_I[[pid],:stop_t,:],np.ones((1,maxT-stop_t,test_I.shape[2]))],axis=1))
    if doPrint:
        print 'Output: ',latent_z_fact.shape,latent_z_cfact.shape,CFAC_I[-1].shape
    
FAC_Z = np.concatenate(FAC_Z,axis=0)
FAC_I = test_I[[k[0] for k in testplist],:,:]
CFAC_Z= np.concatenate(CFAC_Z,axis=0)
CFAC_I= np.concatenate(CFAC_I,axis=0)

print FAC_Z.shape, FAC_I.shape, CFAC_Z.shape, CFAC_I.shape

#: 0
Input:  (1, 5, 48) (1, 5, 8) (1, 5, 4) 5
Inference (1, 5, 30)
Output:  (1, 18, 30) (1, 18, 30) (1, 18, 4)
#: 100
Input:  (1, 6, 48) (1, 6, 8) (1, 6, 4) 6
Inference (1, 6, 30)
Output:  (1, 18, 30) (1, 18, 30) (1, 18, 4)
#: 200
Input:  (1, 4, 48) (1, 4, 8) (1, 4, 4) 4
Inference (1, 4, 30)
Output:  (1, 18, 30) (1, 18, 30) (1, 18, 4)
#: 300
Input:  (1, 4, 48) (1, 4, 8) (1, 4, 4) 4
Inference (1, 4, 30)
Output:  (1, 18, 30) (1, 18, 30) (1, 18, 4)
#: 400
Input:  (1, 3, 48) (1, 3, 8) (1, 3, 4) 3
Inference (1, 3, 30)
Output:  (1, 18, 30) (1, 18, 30) (1, 18, 4)
#: 500
Input:  (1, 2, 48) (1, 2, 8) (1, 2, 4) 2
Inference (1, 2, 30)
Output:  (1, 18, 30) (1, 18, 30) (1, 18, 4)
#: 600
Input:  (1, 4, 48) (1, 4, 8) (1, 4, 4) 4
Inference (1, 4, 30)
Output:  (1, 18, 30) (1, 18, 30) (1, 18, 4)
(666, 18, 30) (666, 18, 4) (666, 18, 30) (666, 18, 4)


In [5]:
#Use the emission distribution to compute mean probabilities
FACT_MEANS  = dkf.emission_fxn(FAC_Z.astype('float32'),I=FAC_I.astype('float32'))
CFACT_MEANS = dkf.emission_fxn(CFAC_Z.astype('float32'),I=CFAC_I.astype('float32'))

print FACT_MEANS[1].shape,CFACT_MEANS[1].shape,len(dataset['patient_dictionary'])

(666, 18, 48) (666, 18, 48) 48


In [6]:
#Collect relevant indices in patient dictionary
#for idx,k in enumerate(dataset['patient_dictionary']):
#    print '(',idx,k,') ',
A1CIdx = range(35,43); 
print [dataset['patient_dictionary'][k] for k in A1CIdx]
GlucIdx = range(43,48);
print [dataset['patient_dictionary'][k] for k in GlucIdx]

['a1c_lt_5.5', 'a1c_lt_6', 'a1c_lt_6.5', 'a1c_lt_7', 'a1c_lt_8', 'a1c_lt_9', 'a1c_lt_10', 'a1c_lt_19']
['gluc_lt_92', 'gluc_lt_102', 'gluc_lt_113', 'gluc_lt_135', 'gluc_lt_989']


In [7]:
#Instead of doing this, look at the 

#First look at A1C levels
FACT_GLUC = FACT_MEANS[1][:,:,GlucIdx]
CFACT_GLUC = CFACT_MEANS[1][:,:,GlucIdx]
FACT_A1C = FACT_MEANS[1][:,:,A1CIdx]
CFACT_A1C = CFACT_MEANS[1][:,:,A1CIdx]
stoplist = [k[1] for k in testplist]

def estimateEffect(FACT,CFACT,stoplist, cutoff = None):
    assert len(stoplist)==FACT.shape[0],'Shape mismatch'
    assert cutoff is not None,'Specify dimension to cut off at for 0/1'
    """NxTxdim <- dim should projected on the simplex """
    factual        = np.zeros(FACT.shape[1])
    counterfactual = np.zeros(CFACT.shape[1])
    for pid, stop_t in enumerate(stoplist):
        data_fact = FACT[pid,stop_t:,:]
        data_cfact= CFACT[pid,stop_t:,:]
        max_fact  = (np.argmax(data_fact,axis=1)>cutoff)*1.
        max_cfact = (np.argmax(data_cfact,axis=1)>cutoff)*1.
        print max_fact, max_cfact

#Normalize along values of A1C
A1C_hist_fact, N_fact, A1C_hist_cfact, N_cfact = estimateEffect(FACT_A1C,CFACT_A1C,stoplist,cutoff = 2)
print A1C_hist_fact
print A1C_hist_cfact

Gluc_hist_fact, N_fact, Gluc_hist_cfact, N_cfact= estimateEffect(FACT_GLUC,CFACT_GLUC,stoplist,cutoff = 2)
print Gluc_hist_fact
print Gluc_hist_cfact
print N_fact,N_cfact

Nmax = 10
T = len(Gluc_hist_fact)

label= 'Proportion of Patients w/ High Glucose'
plt.figure(figsize=(12,8))
data = Gluc_hist_fact[:Nmax]
plt.bar(np.arange(len(data)),data,width=0.3,color='r',hatch='//',label='w/ drugs')
data = Gluc_hist_cfact[:Nmax]
plt.bar(np.arange(len(data))+0.3,data,width=0.3,color='g',hatch='\\',label='w/out drugs')

#plt.ylim([0,0.5])
plt.title(label)
plt.xlabel('Timesteps (Forward Sampling)')
plt.legend()

label= 'Proportion of Patients w/ High A1C'
plt.figure(figsize=(12,8))
data = A1C_hist_fact[:Nmax]
plt.bar(np.arange(len(data)),data,width=0.3,color='r',hatch='//',label='w/ drugs')
data = A1C_hist_cfact[:Nmax]
plt.bar(np.arange(len(data))+0.3,data,width=0.3,color='g',hatch='\\',label='w/out drugs')

#plt.ylim([0,0.5])
plt.title(label)
plt.xlabel('Timesteps (Forward Sampling)')
plt.legend()

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.] [ 0.  0.  0.]
[ 0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.] [ 0.  0.  0.  0.  0.]
[ 0.  0.] [ 0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 1.  0.  0.  0.  0.  0.  1.] [ 0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

TypeError: 'NoneType' object is not iterable

In [ ]:
def plotSamples(samples,patient_dict):
    for idx in range(2):
        row_labels = patient_dict
        data = samples[idx]
        column_labels = range(1,data.shape[0]+1)
        fig, ax = plt.subplots(1,1,figsize=(15,15))
        heatmap = ax.pcolor(data.transpose(), cmap=plt.cm.Blues)
        # put the major ticks at the middle of each cell
        ax.set_xticks(np.arange(data.shape[0])+0.5, minor=False)
        ax.set_yticks(np.arange(data.shape[1])+0.5, minor=False)
        # want a more natural, table-like display
        ax.xaxis.tick_top()
        ax.tick_params(labelsize = 12)
        ax.set_xticklabels(column_labels, minor=False,fontsize=16)
        ax.set_yticklabels(row_labels, minor=False,fontsize=16)
        ax.set_ylim((1,len(row_labels)))

In [ ]:
#First check if the arrow from I->X works
np.random.seed(2)
idxlist = np.random.permutation(10)
small_X = dataset['test_X'][idxlist,:,:]
small_U = dataset['test_U'][idxlist,:,:]
small_I = dataset['test_I'][idxlist,:,:]
z,_,_   = dkf.infer(small_X, U=small_U, I=small_I)

mean_p = dkf.emission_fxn(z.astype('float32'),I=small_I.astype('float32'))
samples = mean_p[1]
plotSamples(samples,patient_dict)

mean_p = dkf.emission_fxn(z.astype('float32'),I=np.ones(small_I.shape).astype('float32'))
samples = mean_p[1]
plotSamples(samples,patient_dict)